# Tree search

## Code

In [1]:
import typing
import math
import collections
import dataclasses
import abc
import itertools

T = typing.TypeVar("T", int, float)
Vector = typing.Tuple[T, T]
VectorComponent = typing.List[T]
Vertex = typing.List[typing.Tuple[T, T]]

def check_axis_intersect(that: Vector, z: T):
    x, y = that
    return y >= z >= x


def is_collision(that: Vertex, other: VectorComponent) -> bool:
    if len(that) != len(other):
        raise ValueError("Invalid arguments, different length of values")

    return all([check_axis_intersect(t, z) for t, z in zip(that, other)])


@dataclasses.dataclass
class VectorNode:
    data: VectorComponent
    value: T

    def __gt__(self, other):
        return self.value > other.value

    def __str__(self):
        return f"{self.data}, {self.value}"

    def __iter__(self):
        return iter([*self.data, self.value])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.vertex[item]

    def __setitem__(self, key, value):
        self.vertex[key] = value


def calc_distance(x: T, y: T):
    return math.fabs(y - x) / 2


def create_axis(_array: Vertex):
    all_data = []

    for arr in _array:
        d = calc_distance(*arr)

        all_data.append(((x, x + d), (x + d, y)))

    return all_data


def create_axis_node(arrays):
    return itertools.product(arrays)


@dataclasses.dataclass
class Node:
    vertex: Vertex
    nodes: list

    def append(self, node: VectorComponent):
        self.nodes.append(node)

    def is_collide(self, node: VectorComponent):
        return is_collision(self.vertex, node)

    def __len__(self):
        return len(self.nodes)

    def __iter__(self):
        return iter(self.nodes)

    def __hash__(self):
        _hash = hash(tuple(self.vertex))
        
        return _hash


@dataclasses.dataclass
class NodeValue:
    vertex: Vertex
    node: VectorNode

    def __gt__(self, other: VectorNode):
        return self.value > other.value

    @property
    def value(self):
        return self.node.value


class NodeContainerInterface(abc.ABC):
    @property
    @abc.abstractmethod
    def axis(self) -> Vertex: ...

    @property
    @abc.abstractmethod
    def children(self): ...

    @property
    @abc.abstractmethod
    def node(self) -> Node: ...

    @property
    @abc.abstractmethod
    def is_parent(self) -> bool: ...

    @abc.abstractmethod
    def insert(self, verx: VectorComponent): ...

    @abc.abstractmethod
    def sort(self): ...

    @abc.abstractmethod
    def __iter__(self): ...


class TreeNode:
    def __init__(self, axis: Vertex, limit_divisions=1):
        self.__children = {}
        self.__node = Node(vertex=axis, nodes=[])
        self.__axis = axis
        self.__limit_divisions = limit_divisions

    def __hash__(self):
        return hash(self.__node)
        
    @property
    def axis(self):
        return [*self.__axis]

    @property
    def children(self):
        return self.__children

    @property
    def node(self) -> Node:
        return self.__node

    @property
    def is_parent(self):
        return len(self.children) != 0

    def insert(self, verx: VectorComponent):
        return self._insert_recursive(verx)

    def sort(self):
        return self._get_iter_child_recursive()

    def __iter__(self):
        return iter(self._get_iter_child_recursive())
        
    def _insert_recursive(self, verx: VectorComponent):
        def create_vertex(verx):
            root_axis = collections.deque()
            
            for axis, c in zip(self.axis, verx):
                x, y = axis
                d = calc_distance(x, y)
                
                if  c >= x and c <= (x + d):
                    root_axis.append((x, x + d))
                else:
                    root_axis.append((x + d, y))

            axis = list(root_axis)

            return axis

        tree = TreeNode(create_vertex(verx), self.__limit_divisions - 1)
        tree_key = hash(tree)
    
        if tree_key in self.__children:
            tree = self.__children[tree_key]
        else:
            self.__children[tree_key] = tree
        
        if self.__limit_divisions > 0:
            return tree.insert(verx)
        else:
            if not tree.node.is_collide(verx):
                raise ValueError(f"Vertex no collide: {tree.node} {verx}")
            
            tree.node.append(verx)
        
        return tree

    def _get_iter_child_recursive(self):
        def get_iter_child(root, nodes=None):
            if nodes is None:
                nodes = []

            for _, child in root.children.items():
                node = child.node
                
                if child.is_parent:
                    get_iter_child(child, nodes)
                else:
                    if len(child.node) > 0:
                        nodes.append(child.node)

            return nodes

        return get_iter_child(self, [])


In [2]:
axis = (-1, 1), (-1, 1)
tree = TreeNode(axis, 100)

tree.insert((.1, .1))
tree.insert((.1, .2))

tree.sort()

[Node(vertex=[(0.09999999999999999, 0.1), (0.09999999999999999, 0.1)], nodes=[(0.1, 0.1)]),
 Node(vertex=[(0.09999999999999999, 0.1), (0.19999999999999998, 0.2)], nodes=[(0.1, 0.2)])]

## Initialize and config data

In [3]:
import sympy as sp
import numpy as np

N_COMPONENTS = 2

syms = sp.symbols(f'x1:{N_COMPONENTS + 1}')

ALL_SYMS = syms
LIMITS_RANGE = -3, 3

def get_n_syms(_range=(-1, 1)):
    return [_range for _ in range(len(syms))]

In [4]:
import operator
import functools

x1 = syms[0]
x2 = syms[1]
# x3 = syms[2]

# eq = sum(syms)
# eq = eq.subs(x1, 1/x1**2).subs(x2, x2**2).subs(x3, x3**3)
eq = functools.reduce(operator.mul, syms, 1)**2 / 100

OBJ_FUNC = sp.lambdify(syms, eq, modules='numpy')

eq

x1**2*x2**2/100

## Execution

In [5]:
import collections
import functools
import itertools

import pandas as pd

def tree_search_algorithm(f, axis, n_it=10, n_p=5, n_limit=0):
    tree = TreeNode(axis)

    all_it = collections.deque()
    all_it_route = collections.deque()

    for it in range(n_it):
        local_min_point = collections.deque()

        for _ in range(n_p):
            data = [np.random.uniform(*_) for _ in tree.axis]
            value = f(*data)

            tree.insert(VectorNode(data, value))

        for container in tree.sort():
            min_node = min(container)

            local_min_point.append(
                NodeValue(container.vertex, min_node)
            )

        if len(local_min_point) <= 0:
            continue

        local_min_node = min(local_min_point)

        all_it.append(list(min(local_min_point).node))
        all_it_route.append(local_min_node.vertex)

        tree = TreeNode(local_min_node.vertex, 2)

    return all_it, all_it_route


all_it, all_it_route = tree_search_algorithm(
    OBJ_FUNC, [(-3, 3) for _ in range(len(syms))], 10, 25
)


In [6]:
pd.DataFrame(all_it_route)

,0,1
0,"(-1.5, 0.0)","(-1.5, 0.0)"
1,"(-0.5625, -0.375)","(-0.1875, 0.0)"
2,"(-0.421875, -0.3984375)","(-0.0234375, 0.0)"
3,"(-0.41015625, -0.4072265625)","(-0.0029296875, 0.0)"
4,"(-0.407958984375, -0.4075927734375)","(-0.0003662109375, 0.0)"
5,"(-0.4077301025390625, -0.407684326171875)","(-4.57763671875e-05, 0.0)"
6,"(-0.40772438049316406, -0.4077186584472656)","(-5.7220458984375e-06, 0.0)"
7,"(-0.40772366523742676, -0.40772294998168945)","(-7.152557373046875e-07, 0.0)"
8,"(-0.4077235758304596, -0.40772348642349243)","(-8.940696716308594e-08, 0.0)"
9,"(-0.4077235087752342, -0.4077234975993633)","(-1.1175870895385742e-08, 0.0)"


In [7]:
pd.DataFrame(all_it)

,0,1,2
0,-0.008959,-6.737469e-01,3.643165e-07
1,-0.440851,-5.198859e-02,5.252911e-06
2,-0.400724,-5.264293e-03,4.450115e-08
3,-0.408076,-3.679499e-04,2.254546e-10
4,-0.407803,-2.046244e-04,6.963289e-11
5,-0.407703,-1.791771e-06,5.336451e-15
6,-0.407724,-3.220481e-06,1.724149e-14
7,-0.407723,-6.299585e-08,6.597124e-18
8,-0.407723,-1.370648e-08,3.123082e-19
9,-0.407724,-7.786514e-10,1.007900e-21
